In [1]:
from jupyter_set_up import init_django

init_django('pictionary')

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from datetime import datetime

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)

        x = torch.randn(28,28).view(-1,1,28,28)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 50)

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = np.prod(x[0].shape)
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [ ]:
def train_X():
    for i, x in enumerate(np.load('train.npy', allow_pickle=True)):
        yield i, torch.tensor(x[1])

def train_y():
    for i, x in enumerate(np.load('train.npy', allow_pickle=True)):
        yield i, torch.tensor(int(x[0]))

def test_X():
    for i, x in enumerate(np.load('test.npy', allow_pickle=True)):
        yield i, torch.tensor(x[1])

def test_y():
    for i, x in enumerate(np.load('test.npy', allow_pickle=True)):
        yield i, torch.tensor(int(x[0]))

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(cnn, train_X, train_y, file):
    BATCH_SIZE = 100
    EPOCHS = 50
    optimizer = optim.Adam(cnn.parameters(), lr=0.01)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(file), BATCH_SIZE)):

            batch_X = []
            batch_y = []

            for j, img in train_X:
                if i<=j<i+BATCH_SIZE:
                    batch_X.append(img)
            for j, img in train_y:
                if i<=j<i+BATCH_SIZE:
                    batch_y.append(img)

            batch_X = torch.stack(batch_X).to(device)
            batch_y = torch.stack(batch_y).to(device)

            cnn.zero_grad()
            outputs = cnn(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backword()
            optimizer.step()
        print("Epoch: {} Loss: {}".format(epoch, loss))


def test(cnn, test_X, test_y, file):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(file))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = cnn(test_X[i].to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: {}".format(round(correct / total, 3)))

In [18]:
start_time = datetime.now()
cnn = CNN().to(device)
train(cnn, train_X(), train_y(), np.load('train.npy', allow_pickle=True))
test(cnn, test_X(), test_y(), np.load('test.npy', allow_pickle=True))
print("Duration: {}".format(datetime.now() - start_time))

  0%|                                                                                                                                                        | 0/18670 [00:00<?, ?it/s]


TypeError: 'generator' object is not subscriptable